# Advent of Code 2020 - Day 4

In [179]:
import kotlin.properties.ReadOnlyProperty
import kotlin.reflect.KProperty

class Passport(
    private val map: Map<String, Any?>
) {
    val birthYear: String by key("byr")
    val issueYear: String by key("iyr")
    val expirationYear: String by key("eyr")
    val height: String by key("hgt")
    val hairColor: String by key("hcl")
    val eyeColor: String by key("ecl")
    val passportId: String by key("pid")
    val countryId: String? by key("cid")

    private fun <T, V> T.key(key: String): ReadOnlyProperty<T, V> =
        object : ReadOnlyProperty<T, V> {
            override fun getValue(thisRef: T, property: KProperty<*>): V = map[key] as V
        }
}



In [180]:
import java.io.File
import java.util.Scanner

val regex = Regex("(\\S+):(\\S+)")

val passports: List<Passport> = Scanner(File("Day4.input.txt"))
    .apply { useDelimiter("${System.lineSeparator()}${System.lineSeparator()}") }
    .use { scanner ->
        generateSequence { if(scanner.hasNext()) scanner.next() else null }
            .map { 
                regex.findAll(it)
                    .associate { it.groupValues[1] to it.groupValues[2] }
                    .let(::Passport)
             }
             .toList()
    }

### Notes

Line separators are OS specific. Turns out that on Windows the separator is `\r\n`. We should use `System.lineSeparator()` to avoid any system-specific issues.

`BufferedReader` can't really target a character easily, so we should use `Scanner` as it accepts a delimiter. We can use `generateSequence` to make the `Scanner` API more functional.

## Part 1

Count the number of valid passports. A passport is valid when all of the required fields are populated (everything except for `countryId`, which is optional.) 

In [181]:
fun Passport.validate(): Result<Passport> = runCatching {
    checkNotNull(birthYear)
    checkNotNull(issueYear)
    checkNotNull(expirationYear)
    checkNotNull(height)
    checkNotNull(hairColor)
    checkNotNull(eyeColor)
    checkNotNull(passportId)
    this
}

passports.count { it.validate().isSuccess }

213

### Notes

This problem is pretty straightforward and most of the work is done during parsing - but there are so many ways to design this solution. I would probably validate the `Passport` within an `init` block in production, but for this problem I wanted to separate the "data processing" part with the "challenge" part. We could also consider a more traditional "mapper" (e.g. `this.x = that.y`) and validate during this process.

Using a delegate could potentially introduce an unexpected null-pointer exception if `validate` wasn't called (and why this function might not want to be separate from the class), but has the added benefit of being able to check for `undefined` vs `null` by inspecting the underlying `Map`.

## Part 2

Perform data validation with the following rules:
- Birth Year - four digits; at least 1920 and at most 2002.
- Issue Year - four digits; at least 2010 and at most 2020.
- Expiration Year - four digits; at least 2020 and at most 2030.
- Height - a number followed by either cm or in:
  - If cm, the number must be at least 150 and at most 193.
  - If in, the number must be at least 59 and at most 76.
- Hair Color - a # followed by exactly six characters 0-9 or a-f.
- Eye Color - exactly one of: amb blu brn gry grn hzl oth.
- Passport ID - a nine-digit number, including leading zeroes.
- Country ID - ignored, missing or not.

Count all the passports that are valid.

In [182]:
fun Passport.validateBirthYear(range: IntRange = 1920..2002): Result<Passport> = runCatching {
    checkNotNull(birthYear)
    check(birthYear.toInt() in range)
    this
}

fun Passport.validateIssueYear(range: IntRange = 2010..2020): Result<Passport> = runCatching {
    checkNotNull(issueYear)
    check(issueYear.toInt() in range)
    this
}

fun Passport.validateExpirationYear(range: IntRange = 2020..2030): Result<Passport> = runCatching {
    checkNotNull(expirationYear)
    check(expirationYear.toInt() in range)
    this
}

fun Passport.validateHeight(metricRange: IntRange = 150..193, imperialRange: IntRange = 59..76): Result<Passport> = runCatching {
    checkNotNull(height)
    check(
        if (height.endsWith("cm")) height.removeSuffix("cm").toInt() in metricRange 
        else if (height.endsWith("in")) height.removeSuffix("in").toInt() in imperialRange
        else false
    )
    this 
}

fun Passport.validateHairColor(regex: Regex = Regex("#([0-9a-fA-F]{3}){2}")): Result<Passport> = runCatching {
    checkNotNull(hairColor)
    check(regex matches hairColor)
    this
}

fun Passport.validateEyeColor(regex: Regex = Regex("amb|blu|brn|gry|grn|hzl|oth")): Result<Passport> = runCatching {
    checkNotNull(eyeColor)
    check(regex matches eyeColor)
    this
}

fun Passport.validatePassportId(regex: Regex = Regex("\\d{9}")): Result<Passport> = runCatching {
    checkNotNull(passportId)
    check(regex matches passportId)
    this
}

fun Passport.validateCountryId(): Result<Passport> = Result.success(this)

fun Passport.strictValidate(): Result<Passport> = runCatching {
    check(validateBirthYear().isSuccess)
    check(validateIssueYear().isSuccess)
    check(validateExpirationYear().isSuccess)
    check(validateHeight().isSuccess)
    check(validateHairColor().isSuccess)
    check(validateEyeColor().isSuccess)
    check(validatePassportId().isSuccess)
    check(validateCountryId().isSuccess)
    this
}

passports.count { it.strictValidate().isSuccess }

147

### Notes

This one was just really tedious and seems to just test my ability to regex (which is poor) and again, should probably be performed a single time during object creation.

Some things to consider is that certain fields could probably be an `inline class` for type-safety, like `hairColor` which could probably be something like `HexColor`. Something like `height` could be `sealed` so that it can denote whether it is metric or imperial. `eyeColor` could even be an `enum`, but using an enum comes with its own concerns.

I suspect that all of these fields could be validated via regex and if I wanted to "code golf" more, I could perhaps define the relationships as a `Pair` of properties to their validator:

In [183]:
val validators = listOf(
    Passport::hairColor to Regex("#([0-9a-fA-F]{3}){2}"),
    Passport::eyeColor to Regex("amb|blu|brn|gry|grn|hzl|oth"),
    Passport::passportId to Regex("\\d{9}"),
)

passports.count { passport ->
    validators.all { it.first(passport) != null && it.second matches it.first(passport) }
}

177